In [1]:
from langchain.memory.buffer_window import ConversationBufferWindowMemory




In [2]:
from langchain.agents import AgentExecutor
from langchain.agents import initialize_agent
from langchain.agents import Tool
from langchain_cohere.react_multi_hop.agent import create_cohere_react_agent
from langchain_core.prompts import ChatPromptTemplate
from langchain_experimental.agents import create_csv_agent
from langchain_groq import ChatGroq

from langchain_community.tools.tavily_search import TavilySearchResults

In [3]:
import os
os.environ['COHERE_API_KEY'] = "AQWILHeT6TLrmqR94NrY4xDqAZSwUp1MmjsSPqV8"

In [4]:

chat = ChatGroq(model="llama3-70b-8192",groq_api_key="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", temperature=0.3)

In [5]:
def file_processor(input=""):
    chat = ChatGroq(model="llama3-70b-8192",groq_api_key="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",temperature=0.3)

    csv_file_path = 'healthcare_source 6.csv'

    agent=create_csv_agent(chat,csv_file_path,allow_dangerous_code=True)
    r=agent.run(input)
    return r
tool1=Tool(
    name="processing_tool",
    func=file_processor,
    description="think yourself as a data analyst. answer to the user request using LLM and create the csv file as per the user request"
)
 

In [6]:
def tavi(input=""):
    tav=TavilySearchResults(api_key="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")
    query=input
    response=tav.search(query)
    if response["results"][0]:
        result=response["results"][0]
        return (result.get("content"))
sear_t=Tool(
    name="search_engine",
    func=tavi,
    description="you are search tool .you need to answer the user query"
)

In [7]:
tools=[tool1,sear_t]

In [8]:
memory=ConversationBufferWindowMemory(memory_key='chat_history',k=3,return_message=True)

In [9]:
  # Call the function to get the actual tools

# Now pass the tools to initialize_agent
agent1 = initialize_agent(
    agent="conversational-react-description",
    tools=tools,
    llm=chat,
    max_iterations=10000,
    verbose=True,
    early_stopping_method="generate",
    memory=memory,
    handle_parsing_errors=True
)

d:\anaconda\envs\langstock\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.3.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(


In [10]:
agent1("Generate a new dataset which only has the first 40 rows of the given dataset, change the order of the column in the new dataset randomly and save it as l1.csv")

d:\anaconda\envs\langstock\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(




> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: processing_tool
Action Input: Generate a new dataset with the first 40 rows of the given dataset, change the order of the columns randomly, and save it as l1.csv

d:\anaconda\envs\langstock\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(



Observation: The new dataset with the first 40 rows of the given dataset, with the columns in a random order, has been saved as l1.csv.
Thought:Do I need to use a tool? No
AI: The task has been completed. The new dataset with the first 40 rows of the given dataset, with the columns in a random order, has been saved as l1.csv.

> Finished chain.


{'input': 'Generate a new dataset which only has the first 40 rows of the given dataset, change the order of the column in the new dataset randomly and save it as l1.csv',
 'chat_history': '',
 'output': 'The task has been completed. The new dataset with the first 40 rows of the given dataset, with the columns in a random order, has been saved as l1.csv.'}